In [1]:
import tensorflow as tf
from scipy import stats
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Generate an underlying permutation matrix.
# Global parameters
#np.random.seed(12)
#Permutation and weight matrices are KxK
sess = tf.Session()
K = 5
num_samples = 100
#Don't have a annealing schedule
tau = tf.constant(0.1)
# Sample a true permutation (in=col, out=row)
perm_true = np.random.permutation(np.arange(K))
P_true = np.zeros((K, K))
P_true[np.arange(K), perm_true] = 1
P_true = np.float32(P_true)
print P_true
#Generate an underlying weight matrix.
W_true = np.float32(np.random.randn(K,K))
print W_true
#Generate ficticious data from the permutation and weight matrix.
#x_i = PWP^T*a_i+epsilon, i.e. x_i~N(PWP^T*a_i,sigma_true) for epsilon~N(0,sigma_true)
noise_scale = 0.1
RChol = np.array(np.random.rand(K,K)*noise_scale,dtype='float32')
sigma_true = RChol.dot(RChol.T)
#Now generate the a_i.
a = np.array(np.random.rand(num_samples,K,1),dtype='float32')
#Now the data x that will be observed.
PWP = P_true.dot(W_true).dot(P_true.T)
x = np.matmul(PWP,a)
x = tf.constant(np.float32(x))
a = tf.constant(np.float32(a))
#Could be a parameter to be learned, but set to constant for now.
sigmaChol = np.array(np.random.rand(K,K)*noise_scale,dtype='float32')

[[ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]]
[[ 0.0226333   0.41999131  0.28158915 -0.47878963 -1.52580702]
 [-0.97597682 -0.51465583 -1.17085767  0.8096934  -0.98757249]
 [ 1.81853771  1.47999811  0.40183362 -0.97915006  1.21385956]
 [-1.5960505  -0.87150091  0.00964934 -0.1828443  -0.67842609]
 [ 0.34980518  0.11603437 -0.78593677 -0.95942044 -2.49864316]]


In [3]:
#Variational model needs to return both a permutation sample P, and a weight sample W
#Weight sample W for now will be independent normals for each Wij.
#P is according to our rounding method.

mu_W = tf.Variable(np.array(np.random.rand(K,K),dtype='float32'))
#Initialise small noise leads to better training results in my experience.
sigma_W = tf.Variable(np.array(np.ones([K,K])*0.01,dtype='float32'))
sigma_W_pos = sigma_W**2
#Define a tensorflow matrix of independence normal random variables for the variational posterior.
q_W = tf.contrib.distributions.Normal(loc=mu_W, scale=sigma_W_pos)

#Now want to define the variational posterior for the permutation.
mu_P = tf.Variable(np.array(np.random.rand(K,K),dtype='float32'))
#Scott's Sinkhorn in logspace
def sinkhorn_logspace(logP, niters=10):
    for _ in range(niters):
        # Normalize columns and take the log again
        logP = logP - tf.reduce_logsumexp(logP, axis=0,keep_dims=True)
        # Normalize rows and take the log again
        logP = logP - tf.reduce_logsumexp(logP, axis=1,keep_dims=True)
    return logP
#Remember to exp() again to get the matrix out of log-space.    
mu_P_sinkhorned = tf.exp(sinkhorn_logspace(tf.log(mu_P),niters=5))
#vectorize the matrix
mu_P_vec = tf.reshape(mu_P_sinkhorned,shape=[-1])
#eta_P is the diagonal of a matrix that we draw permutation samples from.
eta_P = tf.Variable(np.array(np.random.randn(K**2,1),dtype='float32'))
#The bounds that seem to be suggested in Gonzalo's code.
lb = 0.00
ub = 0.01
#Enforcing the bounds.
eta_P_constrained = tf.reshape(lb+(ub-lb)*tf.nn.sigmoid(eta_P),shape=[-1])
#Creating another tensorflow normal distribution according to the model discussed on
#slack.
q_P = tf.contrib.distributions.Normal(loc=mu_P_vec,scale = eta_P_constrained)
def variational_sample():
    #Works by default for the tensorflow distribution object
    W_sample = q_W.sample()
    #Now need a permutation sample. Start by reshaping hte draw from q_P into a KxK matrix
    phi = tf.reshape(q_P.sample(),shape=[K,K])
    #Now have to round using Hungarian. But there is no tensorflow implementation. So we evaluate
    #using sess.run, and run the standard numpy. It's probably better to use "with Session as sess" etc and indent
    #for all this, but that makes it more difficult to use the jupyter notebook cells.
    #So now just generate the sample as in Gonzalo's notebook.
    row, col = optimize.linear_sum_assignment(-sess.run(phi))
    rounded_phi = np.zeros((K, K))
    rounded_phi[row, col] = 1.0
    rounded_phi =  tf.constant(np.float32(rounded_phi))
    P_sample = tau*phi+(1-tau)*rounded_phi
    #Remember returns the sample for both parts of the variational posterior.
    return [W_sample,P_sample]

#This gives a single sample approximation for the expectation of the log prob of q(z|x).
def variational_log_prob(W_sample):
    #Log probability of the sample of W
    var_log_prob = tf.reduce_sum(q_W.log_prob(W_sample))
    #Log probability of the sample of P, just the entropy of the normal from the sampling process. Handy.
    entropy_phi = q_P.entropy()
    return var_log_prob

In [4]:
#Need to create these now to allow for the Hungarian work-around to work. Drawing samples requires the parameters to
#be defined because Hungarian requires actual values. But we have to initialise again later to define the optimization
#related parameters.
sess = tf.Session()
sess.run(tf.global_variables_initializer())
W_sample,P_sample = variational_sample()
#Need to be careful with training algorithm and learning rate. Despite precautions, seem to veer into NaN territory
#at times.
optimizer = tf.train.AdamOptimizer(0.00005)


In [5]:
#logpdf for multivariate normal since tensorflows mvn distribution is currently limited.
def tf_mvn_logpdf(X,Mu,XChol):
    ''' Use this version when X is a vector '''
    Lambda = tf.matrix_inverse(tf.matmul(XChol,tf.transpose(XChol)))
    XMu    = X-Mu
    return_value= (-0.5 * tf.matmul(XMu, tf.matmul(Lambda,tf.transpose(XMu)))
                  +0.5 * tf.log(tf.matrix_determinant(Lambda))
                  -0.5 * np.log(2*np.pi) * int(X.shape[1]))       
    return return_value

In [6]:
#We assume we know both x and a, with both P and W have uniform prior distributions
def generative_log_prob(x,a,P,W,sigmaChol):
    #The following two lines can be used to check inference on each parameter individually.
    #P = tf.constant(P_true)
    #W = tf.constant(W_true)
    #Here x and a are data, but P and W are tensorflow variable samples from the variational posterior
    #In accordance with x_i~N(PWP^T*a_i,sigma_true)
    A = tf.matmul(P,tf.matmul(W,tf.transpose(P)))
    #Below just encodes the prior over the x's. Simply repeatedly call tf_mvn_logpdf for each sample. Can do this
    #quicker for a tf_mvn_logpdf that acts on a matrix of X's rather than a vector, but I didn't have time to reimpl
    #ement int.
    indices = tf.constant(np.arange(num_samples,dtype='int32'))
    def mvn_calls(index):
        return tf_mvn_logpdf(tf.transpose(x[index]),tf.transpose(tf.matmul(A,a[index])),sigmaChol)
    gen_log_prob = tf.reduce_sum(tf.map_fn(mvn_calls,indices,dtype='float32'))
    return gen_log_prob

In [ ]:
#Single sample approximation of the ELBO.
ELBO = generative_log_prob(x,a,P_sample,W_sample,sigmaChol)-variational_log_prob(W_sample)

In [ ]:
train_fn = optimizer.minimize(-ELBO)
train_steps=2000
#Need to run initializer twice to allow for the optimizer to be instantiated. But need to run it earlier in
#order to get around the lack of a Tensorflow hungarian algorithm.
sess.run(tf.global_variables_initializer())
#####
ascent_tracker = np.array([np.nan]*train_steps)
for step in range(train_steps):
    sess.run(train_fn)
    print(step, sess.run([ELBO]))
    ascent_tracker[step]=sess.run(ELBO)
plt.plot(ascent_tracker)


(0, [-1104780.0])
(1, [-1102954.2])
(2, [-1105270.9])
(3, [-1102849.5])
(4, [-1102730.6])
(5, [-1103821.4])
(6, [-1103994.6])
(7, [-1105752.6])
(8, [-1095849.2])
(9, [-1102345.5])
(10, [-1100008.0])
(11, [-1099985.6])
(12, [-1102395.5])
(13, [-1099697.9])
(14, [-1096805.8])
(15, [-1096112.8])
(16, [-1099015.5])
(17, [-1098931.0])
(18, [-1096282.9])
(19, [-1099305.1])
(20, [-1096119.5])
(21, [-1102847.2])
(22, [-1096760.4])
(23, [-1093340.9])
(24, [-1096962.6])
(25, [-1099458.1])
(26, [-1096940.2])
(27, [-1093070.6])
(28, [-1095241.0])
(29, [-1096306.0])
(30, [-1099277.4])
(31, [-1089952.9])
(32, [-1092551.6])
(33, [-1097081.9])
(34, [-1089647.6])
(35, [-1087087.5])
(36, [-1093856.1])
(37, [-1090758.6])
(38, [-1094593.6])
(39, [-1088499.5])
(40, [-1092717.8])
(41, [-1087839.8])
(42, [-1088942.0])
(43, [-1088276.6])
(44, [-1086210.0])
(45, [-1091237.0])
(46, [-1088708.4])
(47, [-1091727.8])
(48, [-1092742.6])
(49, [-1088988.2])
(50, [-1084405.8])
(51, [-1085904.5])
(52, [-1081113.6])
(53